# How to invoke runnables in parallel

To invoke multiple runnables in parallel using LangChain, you can use the RunnableParallel primitive. It allows you to execute multiple independent tasks concurrently and return the results in a dictionary format, with each task's result keyed by its name. Here's a breakdown of how to use RunnableParallel in LangChain, along with a full example.

# Key Concepts
* RunnableParallel: This is a map (dictionary) where each value is a runnable (or something that can be coerced to a runnable, like a function). All tasks will run in parallel and the results will be returned in a dictionary with corresponding keys.

* Chaining: You can chain runnables using the | operator to pass the output of one runnable into the next. In RunnableParallel, each entry runs independently, but you can still manipulate and combine results if needed.

# Example 1: Parallelizing Retrieval and Response Generation

In [ ]:
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import pipeline
import os


# Load environment variables from .env file
load_dotenv(dotenv_path="C:/Users/Admin/Desktop/10-20-2024/.env")

# Initialize the vector store with Hugging Face embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=embedding_model
)

retriever = vectorstore.as_retriever()

# Initialize Hugging Face QA pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased")

# Retrieve relevant context for the question
question = "where did harrison work?"
retrieved_context = retriever.get_relevant_documents(question)

# Prepare the input for the question answering pipeline
context = retrieved_context[0].page_content
qa_input = {
    'context': context,
    'question': question
}

# Directly use Hugging Face QA pipeline for the final answer
response = qa_pipeline(qa_input)

# Print the response
print(response)


c:\Users\Admin\Desktop\10-20-2024\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert

{'score': 0.033392440527677536, 'start': 0, 'end': 8, 'answer': 'harrison'}


# Example 2: Parallel Task Execution (e.g., Jokes and Poems)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from transformers import pipeline

# Initialize Hugging Face pipelines for generating jokes and poems
joke_pipeline = pipeline("text-generation", model="gpt2")  # Use GPT-2 for jokes
poem_pipeline = pipeline("text-generation", model="gpt2")  # Use GPT-2 for poems

# Define chains for joke and poem
joke_chain = lambda input_data: joke_pipeline(f"tell me a joke about {input_data['topic']}")[0]['generated_text']
poem_chain = lambda input_data: poem_pipeline(f"write a 2-line poem about {input_data['topic']}")[0]['generated_text']

# Create a parallel map of tasks
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

# Invoke the parallel chain
response = map_chain.invoke({"topic": "bear"})
print(response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'joke': "tell me a joke about bear attacks on women. And now we know that women are a part of a problem. You should stand up and say'stop it. This is just more of a problem than this is the right way to do it.'", 'poem': "write a 2-line poem about bear and deer together. I'm actually thinking about writing a novel about bear and deer together. I love the idea.\n\nI also love writing novels about bear. The fact that so many young people are being"}


In [ ]:
z